# Setup MediaPipe con Videos Reales del Equipo
**Sistema de Anotación de Video - Entrega 1**

Este notebook permite subir y procesar videos reales grabados por el equipo para entrenar nuestro clasificador de actividades humanas.

## Plan de Grabación del Equipo:
**Total objetivo: 30 videos (10 videos por persona)**

### Distribución por persona:
- **Juan Esteban (P001)**: 10 videos (2 por actividad)
- **Juan David (P002)**: 10 videos (2 por actividad)  
- **Tomas (P003)**: 10 videos (2 por actividad)

### 5 Actividades a grabar:
1. **Caminar hacia la cámara** (15-20 segundos)
2. **Caminar de regreso** (alejándose) (15-20 segundos)
3. **Girar** (rotación completa) (10-15 segundos)
4. **Sentarse** (de pie → sentado) (8-12 segundos)
5. **Ponerse de pie** (sentado → de pie) (8-12 segundos)



In [31]:
!pip install mediapipe opencv-python matplotlib seaborn pandas numpy tqdm


In [32]:
# Verificar instalación
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
from pathlib import Path
from tqdm import tqdm
import json
import shutil
from google.colab import files
from IPython.display import clear_output, HTML, display

print("✅ Todas las dependencias instaladas correctamente")
print(f"📦 MediaPipe version: {mp.__version__}")
print(f"📦 OpenCV version: {cv2.__version__}")

✅ Todas las dependencias instaladas correctamente
📦 MediaPipe version: 0.10.21
📦 OpenCV version: 4.11.0


In [33]:
# Configuración del proyecto con videos reales del equipo
PROJECT_CONFIG = {
    'activities': [
        'caminar_hacia',
        'caminar_regreso',
        'girar',
        'sentarse',
        'ponerse_pie'
    ],
    'team_members': {
        'P001': 'Juan Esteban Ruiz',
        'P002': 'Juan David Quintero',
        'P003': 'Tomas Quintero'
    },
    'video_requirements': {
        'total_target': 30,  # 10 por persona
        'per_person': 10,    # 2 por actividad
        'per_activity': 6,   # 2 por persona × 3 personas
        'format': ['mp4', 'mov', 'avi'],
        'min_duration': 8,   # segundos
        'max_duration': 25,  # segundos
        'min_resolution': '720p'
    },
    'recording_guidelines': {
        'orientation': 'horizontal',
        'lighting': 'buena iluminación natural',
        'background': 'fondo despejado',
        'distance': '2-3 metros de la cámara',
        'stability': 'cámara fija (no mano alzada)'
    },
    'mediapipe_config': {
        'model_complexity': 1,
        'min_detection_confidence': 0.7,
        'min_tracking_confidence': 0.5
    }
}

# Crear estructura de directorios
def create_project_structure():
    """Crear la estructura de directorios del proyecto"""
    dirs = ['data/videos', 'data/landmarks', 'data/metadata', 'data/raw_uploads']

    for dir_path in dirs:
        Path(dir_path).mkdir(parents=True, exist_ok=True)

    for activity in PROJECT_CONFIG['activities']:
        Path(f"data/videos/{activity}").mkdir(parents=True, exist_ok=True)

    print("✅ Estructura de directorios creada")
    print("📁 Carpetas preparadas para cada actividad")

create_project_structure()

print(f"\n👥 EQUIPO DEL PROYECTO:")
for member_id, name in PROJECT_CONFIG['team_members'].items():
    print(f"   {member_id}: {name}")

print(f"\n🎯 OBJETIVO: {PROJECT_CONFIG['video_requirements']['total_target']} videos reales del equipo")


✅ Estructura de directorios creada
📁 Carpetas preparadas para cada actividad

👥 EQUIPO DEL PROYECTO:
   P001: Juan Esteban Ruiz
   P002: Juan David Quintero
   P003: Tomas Quintero

🎯 OBJETIVO: 30 videos reales del equipo


In [34]:
# MOSTRAR INSTRUCCIONES DETALLADAS DE GRABACIÓN
def show_recording_instructions():
    """Mostrar instrucciones completas de grabación"""

    print("📱 INSTRUCCIONES COMPLETAS DE GRABACIÓN")
    print("=" * 60)

    print("\n🎥 CONFIGURACIÓN TÉCNICA:")
    print(f"   📐 Orientación: {PROJECT_CONFIG['recording_guidelines']['orientation'].upper()}")
    print(f"   💡 Iluminación: {PROJECT_CONFIG['recording_guidelines']['lighting']}")
    print(f"   🎭 Fondo: {PROJECT_CONFIG['recording_guidelines']['background']}")
    print(f"   📏 Distancia: {PROJECT_CONFIG['recording_guidelines']['distance']}")
    print(f"   📸 Estabilidad: {PROJECT_CONFIG['recording_guidelines']['stability']}")

    print(f"\n📋 ESPECIFICACIONES TÉCNICAS:")
    print(f"   🎬 Resolución mínima: {PROJECT_CONFIG['video_requirements']['min_resolution']}")
    print(f"   ⏱️ Duración: {PROJECT_CONFIG['video_requirements']['min_duration']}-{PROJECT_CONFIG['video_requirements']['max_duration']} segundos")
    print(f"   📦 Formatos: {', '.join(PROJECT_CONFIG['video_requirements']['format'])}")

    activities_instructions = {
        'caminar_hacia': {
            'setup': 'Persona empieza a 4-5 metros de la cámara',
            'action': 'Camina normalmente hacia la cámara',
            'duration': '15-20 segundos',
            'tips': ['Velocidad natural', 'Mantener dirección recta', 'Terminar a 1 metro de cámara']
        },
        'caminar_regreso': {
            'setup': 'Persona empieza cerca de la cámara (1-2 metros)',
            'action': 'Camina alejándose de la cámara',
            'duration': '15-20 segundos',
            'tips': ['Puede caminar de espaldas o de lado', 'Mantener en el frame', 'Velocidad constante']
        },
        'girar': {
            'setup': 'Persona parada en el centro del frame',
            'action': 'Realiza rotación completa (360°)',
            'duration': '10-15 segundos',
            'tips': ['Giro en el mismo lugar', 'Velocidad moderada', 'Brazos naturales']
        },
        'sentarse': {
            'setup': 'Persona de pie junto a una silla visible',
            'action': 'Se sienta naturalmente',
            'duration': '8-12 segundos',
            'tips': ['Incluir silla en frame', 'Puede repetir 2-3 veces', 'Movimiento natural']
        },
        'ponerse_pie': {
            'setup': 'Persona sentada en silla visible',
            'action': 'Se levanta naturalmente',
            'duration': '8-12 segundos',
            'tips': ['Levantarse completamente', 'Puede repetir 2-3 veces', 'No quedarse agachado']
        }
    }

    print(f"\n📖 INSTRUCCIONES POR ACTIVIDAD:")
    print("=" * 60)

    for activity, instructions in activities_instructions.items():
        print(f"\n🎬 {activity.replace('_', ' ').upper()}:")
        print(f"   📍 Setup: {instructions['setup']}")
        print(f"   🎯 Acción: {instructions['action']}")
        print(f"   ⏱️ Duración: {instructions['duration']}")
        print(f"   💡 Tips:")
        for tip in instructions['tips']:
            print(f"      • {tip}")

    print(f"\n📝 CONVENCIÓN DE NOMBRES:")
    print(f"   P001_caminar_hacia_001.mp4 (Juan Esteban)")
    print(f"   P002_girar_001.mp4 (Juan David)")
    print(f"   P003_sentarse_002.mp4 (Tomas)")
    print(f"   Formato: [PERSONA]_[ACTIVIDAD]_[NUMERO].mp4")

    print(f"\n⚠️ CONSEJOS IMPORTANTES:")
    print(f"   🎥 Grabar en HORIZONTAL siempre")
    print(f"   👤 Una persona por video")
    print(f"   🔇 No importa el audio")
    print(f"   📱 Usar trípode o superficie estable")
    print(f"   ☀️ Evitar contra-luz")
    print(f"   👕 Ropa que contraste con el fondo")

    return activities_instructions

# Mostrar instrucciones
instructions = show_recording_instructions()


📱 INSTRUCCIONES COMPLETAS DE GRABACIÓN

🎥 CONFIGURACIÓN TÉCNICA:
   📐 Orientación: HORIZONTAL
   💡 Iluminación: buena iluminación natural
   🎭 Fondo: fondo despejado
   📏 Distancia: 2-3 metros de la cámara
   📸 Estabilidad: cámara fija (no mano alzada)

📋 ESPECIFICACIONES TÉCNICAS:
   🎬 Resolución mínima: 720p
   ⏱️ Duración: 8-25 segundos
   📦 Formatos: mp4, mov, avi

📖 INSTRUCCIONES POR ACTIVIDAD:

🎬 CAMINAR HACIA:
   📍 Setup: Persona empieza a 4-5 metros de la cámara
   🎯 Acción: Camina normalmente hacia la cámara
   ⏱️ Duración: 15-20 segundos
   💡 Tips:
      • Velocidad natural
      • Mantener dirección recta
      • Terminar a 1 metro de cámara

🎬 CAMINAR REGRESO:
   📍 Setup: Persona empieza cerca de la cámara (1-2 metros)
   🎯 Acción: Camina alejándose de la cámara
   ⏱️ Duración: 15-20 segundos
   💡 Tips:
      • Puede caminar de espaldas o de lado
      • Mantener en el frame
      • Velocidad constante

🎬 GIRAR:
   📍 Setup: Persona parada en el centro del frame
   🎯 Acción:

In [35]:
# SISTEMA DE CARGA ORGANIZADA POR ACTIVIDAD
class ActivityVideoUploader:
    """Subir y organizar videos por actividad"""

    def __init__(self):
        self.uploaded_videos = {activity: [] for activity in PROJECT_CONFIG['activities']}
        self.upload_log = []

    def upload_activity_videos(self, activity):
        """Subir videos para una actividad específica"""
        if activity not in PROJECT_CONFIG['activities']:
            print(f"❌ Actividad '{activity}' no válida")
            return

        print(f"🔼 SUBIR VIDEOS: {activity.replace('_', ' ').upper()}")
        print("=" * 50)

        # Mostrar instrucciones específicas para esta actividad
        self._show_activity_specific_instructions(activity)

        print(f"\n📁 Sube los videos para '{activity.replace('_', ' ')}':")
        print(f"   🎯 Esperados: {PROJECT_CONFIG['video_requirements']['per_activity']} videos")
        print(f"   👥 2 videos por cada miembro del equipo")

        # Subir archivos
        uploaded = files.upload()

        if not uploaded:
            print("❌ No se subieron archivos")
            return

        return self._process_uploaded_files(uploaded, activity)

    def _show_activity_specific_instructions(self, activity):
        """Mostrar instrucciones específicas para una actividad"""
        activity_details = {
            'caminar_hacia': "👤 Persona camina hacia la cámara desde lejos",
            'caminar_regreso': "👤 Persona camina alejándose de la cámara",
            'girar': "🔄 Persona gira 360° en el mismo lugar",
            'sentarse': "💺 Persona se sienta en una silla",
            'ponerse_pie': "🚶 Persona se levanta desde sentado"
        }

        print(f"📖 {activity_details.get(activity, 'Actividad sin descripción')}")

    def _process_uploaded_files(self, uploaded, target_activity):
        """Procesar archivos subidos para una actividad"""
        processed_count = 0
        errors = []

        print(f"\n🗂️ PROCESANDO {len(uploaded)} ARCHIVOS...")

        for filename in uploaded.keys():
            try:
                # Validar formato
                file_ext = filename.lower().split('.')[-1]
                if file_ext not in PROJECT_CONFIG['video_requirements']['format']:
                    errors.append(f"Formato no válido: {filename}")
                    continue

                # Detectar persona del nombre del archivo
                person_id = self._detect_person_from_filename(filename)

                # Generar nombre estandarizado
                video_count = len(self.uploaded_videos[target_activity]) + 1
                if person_id:
                    new_filename = f"{person_id}_{target_activity}_{video_count:03d}.{file_ext}"
                else:
                    new_filename = f"TEAM_{target_activity}_{video_count:03d}.{file_ext}"

                # Mover a carpeta correcta
                dest_dir = Path(f"data/videos/{target_activity}")
                dest_path = dest_dir / new_filename

                shutil.move(filename, str(dest_path))

                # Validar video
                validation_result = self._validate_video(dest_path)

                if validation_result['valid']:
                    self.uploaded_videos[target_activity].append({
                        'filename': new_filename,
                        'original_name': filename,
                        'person_id': person_id,
                        'validation': validation_result
                    })
                    processed_count += 1
                    print(f"   ✅ {filename} → {new_filename}")
                else:
                    errors.append(f"Video inválido: {filename} - {validation_result['reason']}")
                    dest_path.unlink()  # Eliminar archivo inválido

            except Exception as e:
                errors.append(f"Error procesando {filename}: {str(e)}")

        # Mostrar resultados
        print(f"\n📊 RESULTADOS DE CARGA - {target_activity.upper()}:")
        print(f"   ✅ Videos procesados: {processed_count}")
        print(f"   ❌ Errores: {len(errors)}")
        print(f"   📊 Total en actividad: {len(self.uploaded_videos[target_activity])}")

        if errors:
            print(f"\n⚠️ ERRORES ENCONTRADOS:")
            for error in errors[:5]:
                print(f"   • {error}")

        return processed_count

    def _detect_person_from_filename(self, filename):
        """Detectar persona del nombre del archivo"""
        filename_lower = filename.lower()

        # Buscar IDs de persona
        for person_id in PROJECT_CONFIG['team_members'].keys():
            if person_id.lower() in filename_lower:
                return person_id

        # Buscar nombres
        for person_id, name in PROJECT_CONFIG['team_members'].items():
            name_parts = name.lower().split()
            if any(part in filename_lower for part in name_parts):
                return person_id

        return None

    def _validate_video(self, video_path):
        """Validar video básico"""
        try:
            cap = cv2.VideoCapture(str(video_path))

            if not cap.isOpened():
                return {'valid': False, 'reason': 'No se puede abrir el video'}

            # Obtener propiedades
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            duration = frame_count / fps if fps > 0 else 0

            cap.release()

            # Validaciones
            if duration < PROJECT_CONFIG['video_requirements']['min_duration']:
                return {'valid': False, 'reason': f'Muy corto: {duration:.1f}s'}

            if duration > PROJECT_CONFIG['video_requirements']['max_duration']:
                return {'valid': False, 'reason': f'Muy largo: {duration:.1f}s'}

            if width < 640:  # Mínima resolución
                return {'valid': False, 'reason': f'Resolución baja: {width}x{height}'}

            return {
                'valid': True,
                'duration': duration,
                'resolution': f"{width}x{height}",
                'fps': fps
            }

        except Exception as e:
            return {'valid': False, 'reason': f'Error de validación: {str(e)}'}

# Crear instancia del uploader
uploader = ActivityVideoUploader()
print("✅ Sistema de carga por actividad configurado")


✅ Sistema de carga por actividad configurado


In [37]:
# 🚶 SUBIR VIDEOS: CAMINAR HACIA LA CÁMARA
print("🎬 ACTIVIDAD 1/5: CAMINAR HACIA LA CÁMARA")
print("=" * 60)

print("📖 RECORDATORIO PARA ESTA ACTIVIDAD:")
print("   👤 Persona empieza lejos (4-5 metros)")
print("   🚶 Camina normalmente hacia la cámara")
print("   ⏱️ Duración: 15-20 segundos")
print("   🎯 Objetivo: 6 videos (2 por persona)")

print("\n🔼 SUBE TODOS LOS VIDEOS DE 'CAMINAR HACIA' A LA VEZ:")

result_1 = uploader.upload_activity_videos('caminar_hacia')


🎬 ACTIVIDAD 1/5: CAMINAR HACIA LA CÁMARA
📖 RECORDATORIO PARA ESTA ACTIVIDAD:
   👤 Persona empieza lejos (4-5 metros)
   🚶 Camina normalmente hacia la cámara
   ⏱️ Duración: 15-20 segundos
   🎯 Objetivo: 6 videos (2 por persona)

🔼 SUBE TODOS LOS VIDEOS DE 'CAMINAR HACIA' A LA VEZ:
🔼 SUBIR VIDEOS: CAMINAR HACIA
📖 👤 Persona camina hacia la cámara desde lejos

📁 Sube los videos para 'caminar hacia':
   🎯 Esperados: 6 videos
   👥 2 videos por cada miembro del equipo


❌ No se subieron archivos


In [ ]:
# 🚶 SUBIR VIDEOS: CAMINAR DE REGRESO
print("🎬 ACTIVIDAD 2/5: CAMINAR DE REGRESO")
print("=" * 60)

print("📖 RECORDATORIO PARA ESTA ACTIVIDAD:")
print("   👤 Persona empieza cerca (1-2 metros)")
print("   🚶 Camina alejándose de la cámara")
print("   ⏱️ Duración: 15-20 segundos")
print("   🎯 Objetivo: 6 videos (2 por persona)")

print("\n🔼 SUBE TODOS LOS VIDEOS DE 'CAMINAR DE REGRESO' A LA VEZ:")

result_2 = uploader.upload_activity_videos('caminar_regreso')


In [ ]:
# 🔄 SUBIR VIDEOS: GIRAR
print("🎬 ACTIVIDAD 3/5: GIRAR")
print("=" * 60)

print("📖 RECORDATORIO PARA ESTA ACTIVIDAD:")
print("   👤 Persona parada en el centro")
print("   🔄 Gira 360° completo en el mismo lugar")
print("   ⏱️ Duración: 10-15 segundos")
print("   🎯 Objetivo: 6 videos (2 por persona)")

print("\n🔼 SUBE TODOS LOS VIDEOS DE 'GIRAR' A LA VEZ:")

result_3 = uploader.upload_activity_videos('girar')


In [ ]:
# 💺 SUBIR VIDEOS: SENTARSE
print("🎬 ACTIVIDAD 4/5: SENTARSE")
print("=" * 60)

print("📖 RECORDATORIO PARA ESTA ACTIVIDAD:")
print("   👤 Persona de pie junto a silla")
print("   💺 Se sienta naturalmente")
print("   ⏱️ Duración: 8-12 segundos")
print("   🎯 Objetivo: 6 videos (2 por persona)")

print("\n🔼 SUBE TODOS LOS VIDEOS DE 'SENTARSE' A LA VEZ:")

result_4 = uploader.upload_activity_videos('sentarse')


In [38]:
# 🚶 SUBIR VIDEOS: PONERSE DE PIE
print("🎬 ACTIVIDAD 5/5: PONERSE DE PIE")
print("=" * 60)

print("📖 RECORDATORIO PARA ESTA ACTIVIDAD:")
print("   👤 Persona sentada en silla")
print("   🚶 Se levanta completamente")
print("   ⏱️ Duración: 8-12 segundos")
print("   🎯 Objetivo: 6 videos (2 por persona)")

print("\n🔼 SUBE TODOS LOS VIDEOS DE 'PONERSE DE PIE' A LA VEZ:")

result_5 = uploader.upload_activity_videos('ponerse_pie')


🎬 ACTIVIDAD 5/5: PONERSE DE PIE
📖 RECORDATORIO PARA ESTA ACTIVIDAD:
   👤 Persona sentada en silla
   🚶 Se levanta completamente
   ⏱️ Duración: 8-12 segundos
   🎯 Objetivo: 6 videos (2 por persona)

🔼 SUBE TODOS LOS VIDEOS DE 'PONERSE DE PIE' A LA VEZ:
🔼 SUBIR VIDEOS: PONERSE PIE
📖 🚶 Persona se levanta desde sentado

📁 Sube los videos para 'ponerse pie':
   🎯 Esperados: 6 videos
   👥 2 videos por cada miembro del equipo


❌ No se subieron archivos


In [39]:
# RESUMEN FINAL Y PROCESAMIENTO MEDIAPIPE
class MediaPipeProcessor:
    """Procesador MediaPipe para videos del equipo"""

    def __init__(self, config=None):
        if config is None:
            config = PROJECT_CONFIG['mediapipe_config']

        self.mp_pose = mp.solutions.pose
        self.mp_drawing = mp.solutions.drawing_utils
        self.pose = self.mp_pose.Pose(
            model_complexity=config['model_complexity'],
            min_detection_confidence=config['min_detection_confidence'],
            min_tracking_confidence=config['min_tracking_confidence']
        )

        # Landmarks más relevantes para nuestras actividades
        self.relevant_landmarks = [
            11, 12,  # Hombros
            13, 14,  # Codos
            15, 16,  # Muñecas
            23, 24,  # Caderas
            25, 26,  # Rodillas
            27, 28,  # Tobillos
            29, 30, 31, 32  # Pies
        ]

        self.landmark_names = [
            'L_shoulder', 'R_shoulder', 'L_elbow', 'R_elbow',
            'L_wrist', 'R_wrist', 'L_hip', 'R_hip',
            'L_knee', 'R_knee', 'L_ankle', 'R_ankle',
            'L_heel', 'R_heel', 'L_foot', 'R_foot'
        ]

    def process_all_team_videos(self):
        """Procesar todos los videos del equipo"""
        print("⚙️ PROCESAMIENTO MEDIAPIPE - VIDEOS DEL EQUIPO")
        print("=" * 60)

        total_videos = 0
        successful_processing = 0
        processing_results = {}

        # Contar videos totales primero
        for activity in PROJECT_CONFIG['activities']:
            activity_dir = Path(f"data/videos/{activity}")
            if activity_dir.exists():
                video_files = list(activity_dir.glob("*.mp4")) + \
                             list(activity_dir.glob("*.mov")) + \
                             list(activity_dir.glob("*.avi"))
                total_videos += len(video_files)

        print(f"🎬 Videos encontrados: {total_videos}")
        print(f"🎯 Procesando con MediaPipe...")

        # Procesar por actividad
        for activity in PROJECT_CONFIG['activities']:
            activity_dir = Path(f"data/videos/{activity}")

            if not activity_dir.exists():
                continue

            video_files = list(activity_dir.glob("*.mp4")) + \
                         list(activity_dir.glob("*.mov")) + \
                         list(activity_dir.glob("*.avi"))

            if not video_files:
                print(f"⚠️ {activity}: Sin videos")
                continue

            print(f"\n🎬 Procesando {activity.replace('_', ' ').title()}: {len(video_files)} videos")

            activity_results = []

            for video_path in tqdm(video_files, desc=f"{activity}"):
                result = self._extract_landmarks_from_video(video_path)

                if result and result['detection_rate'] > 60:  # Mínimo 60% detección
                    activity_results.append(result)
                    successful_processing += 1

                    # Guardar landmarks
                    self._save_landmarks_csv(result, video_path, activity)

                    print(f"   ✅ {video_path.name}: {result['detection_rate']:.1f}% detección")
                else:
                    print(f"   ❌ {video_path.name}: Detección insuficiente")

            processing_results[activity] = activity_results

        # Resumen final
        print(f"\n📊 PROCESAMIENTO COMPLETADO:")
        print(f"   🎥 Videos totales: {total_videos}")
        print(f"   ✅ Procesados exitosamente: {successful_processing}")
        print(f"   📈 Tasa de éxito: {(successful_processing/total_videos)*100:.1f}%")

        # Guardar resumen
        self._save_processing_summary(processing_results, total_videos, successful_processing)

        return processing_results

    def _extract_landmarks_from_video(self, video_path):
        """Extraer landmarks de un video"""
        cap = cv2.VideoCapture(str(video_path))

        if not cap.isOpened():
            return None

        landmarks_sequence = []
        frame_count = 0
        detection_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            results = self._process_frame(frame)
            frame_count += 1

            if results.pose_landmarks:
                detection_count += 1
                landmarks_data = []
                for idx in self.relevant_landmarks:
                    landmark = results.pose_landmarks.landmark[idx]
                    landmarks_data.extend([
                        landmark.x, landmark.y, landmark.z, landmark.visibility
                    ])
                landmarks_sequence.append(landmarks_data)
            else:
                landmarks_sequence.append([np.nan] * (len(self.relevant_landmarks) * 4))

        cap.release()

        detection_rate = (detection_count / frame_count) * 100 if frame_count > 0 else 0

        return {
            'landmarks': np.array(landmarks_sequence),
            'frames_total': frame_count,
            'frames_detected': detection_count,
            'detection_rate': detection_rate,
            'video_path': str(video_path)
        }

    def _process_frame(self, frame):
        """Procesar frame con MediaPipe"""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(rgb_frame)
        return results

    def _save_landmarks_csv(self, result, video_path, activity):
        """Guardar landmarks como CSV"""
        landmarks_df = pd.DataFrame(
            result['landmarks'],
            columns=[f"{name}_{coord}" for name in self.landmark_names
                    for coord in ['x', 'y', 'z', 'visibility']]
        )

        # Agregar metadata
        landmarks_df['activity'] = activity
        landmarks_df['video_file'] = video_path.name
        landmarks_df['frame_number'] = range(len(landmarks_df))
        landmarks_df['detection_rate'] = result['detection_rate']

        csv_path = Path(f"data/landmarks/{video_path.stem}_landmarks.csv")
        landmarks_df.to_csv(csv_path, index=False)

    def _save_processing_summary(self, results, total, successful):
        """Guardar resumen de procesamiento"""
        summary = {
            'processing_date': datetime.now().isoformat(),
            'team_members': PROJECT_CONFIG['team_members'],
            'total_videos': total,
            'successful_processing': successful,
            'success_rate': (successful/total)*100 if total > 0 else 0,
            'results_by_activity': {
                activity: {
                    'videos_processed': len(activity_results),
                    'avg_detection_rate': np.mean([r['detection_rate'] for r in activity_results]) if activity_results else 0,
                    'total_frames': sum([r['frames_total'] for r in activity_results]) if activity_results else 0
                }
                for activity, activity_results in results.items()
            }
        }

        with open('data/metadata/team_processing_summary.json', 'w') as f:
            json.dump(summary, f, indent=2)

# Crear procesador y ejecutar
processor = MediaPipeProcessor()

def run_final_processing():
    """Ejecutar procesamiento final completo"""
    print("🏁 PROCESAMIENTO FINAL DEL DATASET DEL EQUIPO")
    print("=" * 60)

    # Mostrar resumen de videos cargados
    print("\n📊 RESUMEN DE VIDEOS CARGADOS:")
    total_loaded = 0
    for activity in PROJECT_CONFIG['activities']:
        activity_dir = Path(f"data/videos/{activity}")
        if activity_dir.exists():
            video_count = len(list(activity_dir.glob("*.*")))
            total_loaded += video_count
            print(f"   🎬 {activity.replace('_', ' ').title()}: {video_count} videos")

    print(f"\n📈 TOTAL CARGADO: {total_loaded} videos")
    print(f"🎯 META: {PROJECT_CONFIG['video_requirements']['total_target']} videos")
    print(f"📊 Progreso: {(total_loaded/PROJECT_CONFIG['video_requirements']['total_target'])*100:.1f}%")

    if total_loaded > 0:
        print(f"\n⚙️ INICIANDO PROCESAMIENTO MEDIAPIPE...")
        results = processor.process_all_team_videos()

        print(f"\n🎉 ¡ENTREGA 1 COMPLETADA!")
        print(f"   📁 Videos: data/videos/")
        print(f"   📊 Landmarks: data/landmarks/")
        print(f"   📋 Metadata: data/metadata/")
        print(f"   ✅ Listo para EDA")

        return results
    else:
        print(f"\n⚠️ No se encontraron videos para procesar")
        return None

print("✅ Procesador MediaPipe configurado")
print("🚀 Ejecuta: run_final_processing() después de subir todos los videos")


✅ Procesador MediaPipe configurado
🚀 Ejecuta: run_final_processing() después de subir todos los videos


## ✅ Checklist de Carga de Videos

### Progreso por Actividad:
- [ ] **Caminar hacia** (6 videos objetivo)
- [ ] **Caminar regreso** (6 videos objetivo)
- [ ] **Girar** (6 videos objetivo)
- [ ] **Sentarse** (6 videos objetivo)
- [ ] **Ponerse de pie** (6 videos objetivo)

### Distribución por Miembro:
- [ ] **Juan Esteban (P001)**: 10 videos (2 por actividad)
- [ ] **Juan David (P002)**: 10 videos (2 por actividad)
- [ ] **Tomas (P003)**: 10 videos (2 por actividad)

### Pasos Finales:
1. **Ejecutar todas las celdas de carga** (6-10)
2. **Verificar que se cargaron todos los videos**
3. **Ejecutar**: `run_final_processing()`
4. **Resultado**: Dataset completo procesado

---
**Estado**: Videos reales del equipo listos para análisis


In [40]:
# 🚀 EJECUTAR PROCESAMIENTO FINAL
print("🎬 EJECUTANDO PROCESAMIENTO FINAL...")
final_results = run_final_processing()

if final_results:
    print("\n✅ DATASET DEL EQUIPO COMPLETADO")
    print("🎯 Continuar con: 02_eda_inicial.ipynb")
else:
    print("\n📤 Pendiente: Cargar videos en celdas anteriores")


🎬 EJECUTANDO PROCESAMIENTO FINAL...
🏁 PROCESAMIENTO FINAL DEL DATASET DEL EQUIPO

📊 RESUMEN DE VIDEOS CARGADOS:
   🎬 Caminar Hacia: 13 videos
   🎬 Caminar Regreso: 13 videos
   🎬 Girar: 13 videos
   🎬 Sentarse: 13 videos
   🎬 Ponerse Pie: 13 videos

📈 TOTAL CARGADO: 65 videos
🎯 META: 30 videos
📊 Progreso: 216.7%

⚙️ INICIANDO PROCESAMIENTO MEDIAPIPE...
⚙️ PROCESAMIENTO MEDIAPIPE - VIDEOS DEL EQUIPO
🎬 Videos encontrados: 65
🎯 Procesando con MediaPipe...

🎬 Procesando Caminar Hacia: 13 videos


caminar_hacia:   0%|          | 0/13 [00:08<?, ?it/s]


KeyboardInterrupt: 